In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import seaborn as sns
import math
import re
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('final_df.csv')

# Feature Engineering

In [3]:
df = df.dropna(subset=['weather_temperature'])
df = df.drop(columns=['weather_humidity','weather_detail','stadium_address','stadium_weather_station_code',
                      'stadium_weather_type','STATION','ELEVATION'])
df.drop(columns=['score_home','score_away', 'stadium_location','total'],inplace=True)
df.drop('schedule_date', axis=1,inplace=True)

df['over_under_result'] = np.where(df['over_under_result'] == 'Over',1,0)
df['stadium_surface'] = np.where(df['stadium_surface'] == 'Grass',1,0)
df['schedule_playoff'] = list(map(int, df['schedule_playoff']))
df['stadium_neutral'] = list(map(int, df['stadium_neutral']))
df = pd.get_dummies(df, columns = ['stadium_type'], drop_first= True)
df = pd.get_dummies(df, columns = ['stadium_name'], drop_first= True)
df = pd.get_dummies(df, columns = ['team_home'], drop_first= True)
df = pd.get_dummies(df, columns = ['team_away'], drop_first= True)
df = pd.get_dummies(df, columns = ['team_favorite_id'], drop_first= True)

uint8_columns = list(df.select_dtypes('uint8').columns)

for colname in df.columns:
    if colname in uint8_columns:
        df[colname] = df[colname].astype(int)

# Feature Selection

In [4]:
# Split data to be used in the models
# Create matrix of features
X = df.drop('over_under_result', axis=1) # grabs everything else but 'Default'

# Create target variable
y = df['over_under_result'] # y is the column we're trying to predict

In [7]:
corrs_dict={}
for col in X.columns:
    corr=y.corr(X[col])
    corrs_dict[corr]=col
    
sorted(corrs_dict.items(),reverse=True)

[(0.02643507765088869, 'stadium_name_Cinergy Field'),
 (0.026416287574825382, 'stadium_name_New Era Field'),
 (0.023824693598373855, 'team_away_San Diego Chargers'),
 (0.02267698602796424, 'team_favorite_id_GB'),
 (0.022233434271607625, 'team_home_Los Angeles Rams'),
 (0.021686128129850147, 'team_home_Tennessee Titans'),
 (0.020461361803612396, 'stadium_name_Mile High Stadium'),
 (0.020380118175394576, 'stadium_name_Nissan Stadium'),
 (0.019866963896616848, 'team_away_Indianapolis Colts'),
 (0.019401942995838185, 'team_favorite_id_DET'),
 (0.019259995276488473, 'schedule_playoff'),
 (0.01905794388098324, 'stadium_name_Anaheim Stadium'),
 (0.018970180894961983, 'team_favorite_id_LAR'),
 (0.018115757195356234, 'team_away_Green Bay Packers'),
 (0.016021391998387598, 'team_favorite_id_DEN'),
 (0.01547820305912302, 'team_favorite_id_TEN'),
 (0.015331177015877535, 'team_home_Denver Broncos'),
 (0.015285025033512398, 'team_home_Detroit Lions'),
 (0.014664346586771577, 'team_home_New Orleans S

In [8]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
def vif_(X):
    vif = pd.DataFrame()
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    vif['Features'] = X.columns
    return vif
Vif = vif_(X)

Vif.sort_values('VIF',ascending=False)[:5]

/Users/alexzieky/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,VIF,Features
99,inf,team_home_Houston Oilers
75,inf,stadium_name_Sun Devil Stadium
73,inf,stadium_name_Sports Authority Field at Mile High
72,inf,stadium_name_Soldier Field
71,inf,stadium_name_Seattle Kingdome


# Modeling